# Imports

In [2]:
import inseq
import warnings

warnings.filterwarnings("ignore")

# Code

In [3]:
class InseqAttributer:
    def __init__(self, model="./models/test/", attribution_method="input_x_gradient") -> None:
        """Create an object that loads a given model through Inseq with a given attribution method"""
        self.model = inseq.load_model(model, attribution_method)
    
    def attribute(self, inp_out:list):
        """Use the Inseq model to generate feature attributions using a given input or input & output"""
        out = self.model.attribute(*inp_out,
                                   attribute_target=True,
                                   step_scores=["probability"])
        out.show()

# Testing

## Inseq testing

In [18]:
inseq_testing = InseqAttributer("./models/test/", "input_x_gradient")

In [19]:
inseq_testing.attribute(["OlympiadeN;NOM;FEM;PL",
                         "Olympiaden"])

Attributing with input_x_gradient...: 100%|██████████| 12/12 [00:06<00:00,  1.77it/s]


,O,l,y,m,p,i,a,d,e,n,</s>
O,0.115,0.108,0.047,0.059,0.044,0.066,0.095,0.019,0.084,0.029,0.01
l,0.154,0.068,0.043,0.053,0.023,0.017,0.034,0.018,0.025,0.017,0.023
y,0.037,0.067,0.055,0.07,0.033,0.016,0.023,0.014,0.014,0.014,0.015
m,0.109,0.075,0.119,0.091,0.036,0.036,0.057,0.03,0.068,0.021,0.017
p,0.023,0.051,0.071,0.075,0.062,0.037,0.055,0.052,0.023,0.022,0.031
i,0.046,0.035,0.027,0.044,0.064,0.025,0.035,0.039,0.064,0.052,0.075
a,0.019,0.03,0.035,0.042,0.144,0.17,0.051,0.058,0.046,0.059,0.107
d,0.101,0.075,0.077,0.055,0.06,0.106,0.076,0.134,0.113,0.076,0.057
e,0.04,0.012,0.01,0.042,0.039,0.051,0.049,0.029,0.054,0.021,0.02
N,0.028,0.017,0.03,0.019,0.026,0.026,0.019,0.04,0.056,0.059,0.023


In [20]:
inseq_testing.attribute(["SkelettN;NOM;NEUT;PL",
                         "Skelette"])

Attributing with input_x_gradient...: 100%|██████████| 10/10 [00:05<00:00,  1.73it/s]


,S,k,e,l,e,t,t,e,</s>
S,0.091,0.089,0.068,0.062,0.063,0.04,0.062,0.031,0.07
k,0.173,0.094,0.058,0.032,0.042,0.015,0.023,0.017,0.013
e,0.046,0.022,0.03,0.025,0.026,0.011,0.015,0.012,0.029
l,0.046,0.022,0.068,0.056,0.079,0.115,0.025,0.05,0.02
e,0.016,0.026,0.034,0.036,0.048,0.034,0.059,0.012,0.031
t,0.107,0.157,0.117,0.179,0.062,0.143,0.101,0.088,0.037
t,0.036,0.036,0.062,0.047,0.057,0.021,0.053,0.041,0.031
N,0.054,0.013,0.037,0.042,0.045,0.053,0.101,0.054,0.073
;,0.032,0.035,0.03,0.037,0.036,0.047,0.042,0.024,0.045
N,0.019,0.017,0.013,0.017,0.017,0.019,0.02,0.012,0.02


In [21]:
inseq_testing.attribute(["WunschN;NOM;MASC;PL"])

Attributing with input_x_gradient...: 100%|██████████| 9/9 [00:04<00:00,  1.80it/s]


,W,u,n,s,c,h,e,</s>
W,0.092,0.048,0.079,0.051,0.033,0.021,0.043,0.022
u,0.07,0.075,0.077,0.069,0.036,0.055,0.046,0.012
n,0.036,0.057,0.043,0.022,0.026,0.077,0.036,0.077
s,0.045,0.081,0.06,0.066,0.048,0.088,0.02,0.031
c,0.095,0.037,0.036,0.035,0.029,0.064,0.034,0.035
h,0.071,0.055,0.028,0.04,0.026,0.03,0.044,0.034
N,0.024,0.038,0.025,0.043,0.06,0.031,0.102,0.021
;,0.047,0.033,0.041,0.048,0.042,0.039,0.061,0.063
N,0.021,0.053,0.045,0.042,0.056,0.04,0.065,0.019
O,0.03,0.034,0.032,0.037,0.031,0.047,0.024,0.02


In [22]:
inseq_testing.attribute(["KastanieN;NOM;FEM;PL"])

Attributing with input_x_gradient...: 100%|██████████| 11/11 [00:05<00:00,  1.80it/s]


,K,a,s,t,a,n,i,e,n,</s>
K,0.116,0.043,0.042,0.032,0.041,0.036,0.062,0.05,0.043,0.017
a,0.086,0.052,0.065,0.018,0.02,0.037,0.044,0.04,0.02,0.027
s,0.041,0.055,0.025,0.023,0.02,0.041,0.017,0.015,0.012,0.01
t,0.092,0.226,0.086,0.063,0.095,0.061,0.068,0.061,0.128,0.093
a,0.025,0.039,0.097,0.125,0.066,0.032,0.142,0.08,0.015,0.009
n,0.11,0.016,0.158,0.084,0.177,0.072,0.032,0.046,0.02,0.017
i,0.053,0.062,0.123,0.093,0.029,0.069,0.107,0.071,0.041,0.023
e,0.029,0.061,0.027,0.064,0.029,0.022,0.041,0.034,0.022,0.037
N,0.019,0.017,0.022,0.032,0.015,0.04,0.02,0.065,0.07,0.023
;,0.04,0.031,0.035,0.033,0.028,0.045,0.036,0.033,0.037,0.041


In [23]:
inseq_testing.attribute(["DialektN;NOM;MASC;PL"])

Attributing with input_x_gradient...: 100%|██████████| 10/10 [00:05<00:00,  1.76it/s]


,D,i,a,l,e,k,t,e,</s>
D,0.156,0.108,0.057,0.056,0.03,0.047,0.031,0.039,0.036
i,0.085,0.085,0.043,0.031,0.019,0.04,0.017,0.024,0.022
a,0.092,0.043,0.088,0.101,0.048,0.087,0.067,0.122,0.014
l,0.117,0.042,0.074,0.027,0.028,0.048,0.125,0.068,0.045
e,0.07,0.044,0.096,0.059,0.123,0.098,0.099,0.06,0.037
k,0.013,0.017,0.026,0.024,0.037,0.027,0.019,0.016,0.02
t,0.011,0.056,0.039,0.078,0.104,0.116,0.075,0.018,0.045
N,0.046,0.036,0.042,0.036,0.05,0.047,0.064,0.023,0.031
;,0.024,0.025,0.034,0.027,0.04,0.023,0.027,0.019,0.049
N,0.014,0.033,0.021,0.024,0.025,0.024,0.017,0.007,0.017
